In [3]:
from __future__ import division
from keras.models import load_model
import csv
import copy
import numpy as np
import distance
from jellyfish._jellyfish import damerau_levenshtein_distance
import csv
from sklearn import metrics
from math import sqrt
import time
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from collections import Counter

In [5]:
eventlog = 'contextualised_full_context_process_event_data_iot_context_full_reworked.csv'
csvfile = open('data/%s' % eventlog, 'r')
datareader = csv.reader(csvfile, delimiter=',', quotechar='|')

#helper variables
lastcase = ''
firstLine = True
casestarttime = None
lasteventtime = None

#lists we need to save all the information
lineseqs = []
timeseqs = []
timeseqs2 = []
timeseqs3 = []

lines = []
caseids = []
times = []
times2 = []
times3 = []
times4 = []

numlines = 0 # to count the number of lines, to know the elements per fold

next(datareader, None)  # skip the headers
for row in datareader: #the columns are "CaseID,ActivityID,CompleteTimestamp"
    t = time.strptime(row[2], "%Y-%m-%d %H:%M:%S") #creates a datetime object from row[2]
    if row[0]!=lastcase:  #'lastcase' is to save the last executed case for the loop
        casestarttime = t
        lasteventtime = t
        lastcase = row[0]
        if not firstLine:
            lines.append('!')
            lineseqs.append(lines) # the activities list
            timeseqs.append(times) # the timeseq lists
            timeseqs2.append(times2) 
            timeseqs3.append(times3)
        caseids = []
        lines = []
        times = []  # for the timesincelastevent
        times2 = [] # for the timesincecasestart
        times3 = [] # saves the actual time
        numlines+=1 # to counter the number of lines
        
    timesincelastevent = datetime.fromtimestamp(time.mktime(t))-datetime.fromtimestamp(time.mktime(lasteventtime))
    timesincecasestart = datetime.fromtimestamp(time.mktime(t))-datetime.fromtimestamp(time.mktime(casestarttime))
    midnight = datetime.fromtimestamp(time.mktime(t)).replace(hour=0, minute=0, second=0, microsecond=0)
    timesincemidnight = datetime.fromtimestamp(time.mktime(t))-midnight
    timediff = 86400 * timesincelastevent.days + timesincelastevent.seconds     #multiply with 60*60*24 = 86400 to go from days to seconds
    timediff2 = 86400 * timesincecasestart.days + timesincecasestart.seconds    #the .seconds method gives the time in seconds
    timediff3 = datetime.fromtimestamp(time.mktime(t)).weekday() #day of the week
    caseids.append(row[0])
    lines.append(str(row[1])) #add the activity label to the line list
    times.append(timediff)
    times2.append(timediff2)
    times3.append(datetime.fromtimestamp(time.mktime(t)))
    lasteventtime = t
    firstLine = False

# add last case
lines.append('!')
lineseqs.append(lines) # lists of lists contains the traces of activities per case
timeseqs.append(times) # timesincelastevent
timeseqs2.append(times2) # timesincecasestart
timeseqs3.append(times3) # the actual time
numlines+=1

divisor = np.mean([item for sublist in timeseqs for item in sublist])
print('divisor: {}'.format(divisor))
divisor2 = np.mean([item for sublist in timeseqs2 for item in sublist])
print('divisor2: {}'.format(divisor2))

divisor: 2636.3360541044776
divisor2: 303966.8337553305


In [6]:
print('There are', len(lineseqs), 'cases')

There are 157 cases


In [7]:
maxlen = max(map(lambda x: len(x),lineseqs))
chars = map(lambda x : set(x),lineseqs)
chars = list(set().union(*chars))
target_chars = copy.copy(chars)
chars.remove('!')
# Custom sorting function to convert strings to integers for numerical sorting
def custom_sort(item):
    return int(item)

# Sorting the list using the custom sorting function
chars = sorted(chars, key=custom_sort)

print('total chars: {}, target chars: {}'.format(len(chars), len(target_chars)))
print('maxlen', maxlen)
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
target_char_indices = dict((c, i) for i, c in enumerate(target_chars))
target_indices_char = dict((i, c) for i, c in enumerate(target_chars))
print(indices_char)

total chars: 51, target chars: 52
maxlen 635
{0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9', 10: '10', 11: '11', 12: '12', 13: '13', 14: '14', 15: '15', 16: '16', 17: '17', 18: '18', 19: '19', 20: '20', 21: '21', 22: '22', 23: '23', 24: '24', 25: '25', 26: '26', 27: '27', 28: '28', 29: '29', 30: '30', 31: '31', 32: '32', 33: '33', 34: '34', 35: '35', 36: '36', 37: '37', 38: '38', 39: '39', 40: '40', 41: '41', 42: '42', 43: '43', 44: '44', 45: '45', 46: '46', 47: '47', 48: '48', 49: '49', 50: '50'}


In [8]:
# set parameters
predict_size = 1

# load model, set this to the model generated by train.py
model = load_model("output_files\models\model_78-1.11.h5")

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'output_files\models\model_78-1.11.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [9]:

"""
To convert the trace activities (sentence) with associated time-related information (times and times3) into a format suitable for the LSTM

sentence: The sequence of activities
times: A list of time differences between consecutive events.
times3: A list of timestamps for each event.
maxlen: Maximum length of the sequence 

X: A 3D NumPy array representing the encoded sequence with additional features.
"""

def encode(sentence, times, times3, maxlen=maxlen):
    num_features = len(chars)+5 # encodes an extra 5 features
    X = np.zeros((1, maxlen, num_features), dtype=np.float32) # the encoded batch has shape [1, maxlen, num_features]
    leftpad = maxlen-len(sentence) 
    times2 = np.cumsum(times) # returns the cumulative sum of the elements along a given axis
    for t, char in enumerate(sentence): # Iterate over each event (char) and its index (t) in the input sequence.
        print('char', char, 't', t)
        midnight = times3[t].replace(hour=0, minute=0, second=0, microsecond=0)
        timesincemidnight = times3[t]-midnight
        multiset_abstraction = Counter(sentence[:t+1])
        for c in chars:
            if c==char:
                X[0, t+leftpad, char_indices[c]] = 1
        X[0, t+leftpad, len(chars)] = t+1 # The position of the event in the sequence (t+1).
        X[0, t+leftpad, len(chars)+1] = times[t]/divisor # The normalized time difference (times[t]/divisor).
        X[0, t+leftpad, len(chars)+2] = times2[t]/divisor2 # The normalized cumulative time difference (times2[t]/divisor2).
        X[0, t+leftpad, len(chars)+3] = timesincemidnight.seconds/86400 # The normalized time since midnight (timesincemidnight.seconds/86400).
        X[0, t+leftpad, len(chars)+4] = times3[t].weekday()/7 # The normalized day of the week (times3[t].weekday()/7).
    return X

def getSymbol(predictions):
    maxPrediction = 0
    symbol = ''
    i = 0;
    print(predictions)
    for prediction in predictions:
        if(prediction>=maxPrediction):
            maxPrediction = prediction
            symbol = target_indices_char[i]
        i += 1
    return symbol

In [10]:
# make predictions
with open('output_files/results/next_activity_and_time_%s' % eventlog, 'w') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(["CaseID", "Prefix length", "Groud truth", "Predicted", "Levenshtein", "Damerau", "Jaccard", "Ground truth times", "Predicted times", "RMSE", "MAE"])
    for prefix_size in range(2,maxlen):
        print('prefix size', prefix_size)
        for line, caseid, times, times3 in zip(lineseqs, caseids, timeseqs, timeseqs3):
            print('check')
            times.append(0)
            cropped_line = line[:prefix_size]
            cropped_times = times[:prefix_size]
            cropped_times3 = times3[:prefix_size]
            print('cropped line', cropped_line)
   
            if '!' in cropped_line:
                continue # make no prediction for this case, since this case has ended already
            ground_truth = line[prefix_size:prefix_size+predict_size]
            ground_truth_t = times[prefix_size:prefix_size+predict_size]
            predicted = ''
            predicted_t = []
            for i in range(predict_size):
                if len(ground_truth)<=i:
                    continue
                
                enc = encode(cropped_line, cropped_times, cropped_times3)
                print(enc.shape)

                y = model.predict(enc, verbose=0)
                print(y)
                y_char = y[0]
                print('ychar', y_char)
                prediction = getSymbol(y_char)              
                cropped_line += prediction
            
                if prediction == '!': # end of case was just predicted, therefore, stop predicting further into the future
                    print('! predicted, end case')
                    break
                predicted += prediction
            output = []
            if len(ground_truth)>0:
                output.append(caseid)
                output.append(prefix_size)
                output.append(str(ground_truth).encode("utf-8"))
                output.append(str(predicted).encode("utf-8"))
                output.append(1 - distance.nlevenshtein(predicted, ground_truth))
                dls = 1 - (damerau_levenshtein_distance(str(predicted), str(ground_truth)) / max(len(predicted),len(ground_truth)))
                if dls<0:
                    dls=0 # we encountered problems with Damerau-Levenshtein Similarity on some linux machines where the default character encoding of the operating system caused it to be negative, this should never be the case
                output.append(dls)
                output.append(1 - distance.jaccard(predicted, ground_truth))
                output.append('; '.join(str(x) for x in ground_truth_t))
                output.append('; '.join(str(x) for x in predicted_t))
                if len(predicted_t)>len(ground_truth_t): # if predicted more events than length of case, only use needed number of events for time evaluation
                    predicted_t = predicted_t[:len(ground_truth_t)]
                if len(ground_truth_t)>len(predicted_t): # if predicted less events than length of case, put 0 as placeholder prediction
                    predicted_t.extend(range(len(ground_truth_t)-len(predicted_t)))
                if len(ground_truth_t)>0 and len(predicted_t)>0:
                    output.append('')
                    output.append(metrics.mean_absolute_error([ground_truth_t[0]], [predicted_t[0]]))
                    #output.append(metrics.median_absolute_error([ground_truth_t[0]], [predicted_t[0]]))
                else:
                    output.append('')
                    output.append('')
                    output.append('')
                spamwriter.writerow(output)

FileNotFoundError: [Errno 2] No such file or directory: 'output_files/results/next_activity_and_time_contextualised_full_context_process_event_data_iot_context_full_reworked.csv'

In [ ]:
line = ['10', '1', '14', '13', '15', '14', '15', '9', '7', '4', '14', '15', '14', '5', '15', '2', '4', '14', '15', '14', '5', '3', '13', '8', '6', '14', '15', '15', '14', '14', '15', '12', '!']

In [27]:
line[:2]

['10', '1']

In [ ]:
line[:prefix_size]